<a href="https://colab.research.google.com/github/natalievolk/LearnAI/blob/main/LearnAI_Take_Five.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader,TensorDataset
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import cv2
import requests
from io import BytesIO
from tqdm.notebook import tqdm

In [ ]:
train_txt = "https://raw.githubusercontent.com/natalievolk/LearnAI/main/updated_train.txt"
test_txt = "https://raw.githubusercontent.com/natalievolk/LearnAI/main/updated_test.txt"

In [ ]:
class ImagesDataset(Dataset):
    def __init__(self, text_file, transform):
        self.df = pd.read_csv(text_file, sep=' ')
        self.df.columns = ['jpg_url', 'classification']
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        response = requests.get(self.df['jpg_url'][idx])
        img = Image.open(BytesIO(response.content)).resize((225, 225))
        pix = np.array(img)
        try:
          pix = self.transform(pix)
        except:
          pix = np.stack((np.array(pix, copy=True), np.array(pix, copy=True), np.array(pix, copy=True)), axis=2)
          pix = self.transform(pix)
 
        return pix, int(self.df['classification'][idx])

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = ImagesDataset(train_txt,transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, num_workers=16)

test_dataset = ImagesDataset(test_txt,transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, num_workers=16)

In [ ]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2,2)
        self.conv1 = nn.Conv2d(3,8,5)
        self.conv2 = nn.Conv2d(8,32,7)
        self.conv3 = nn.Conv2d(32,64,9)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64*22*22, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 5)
        self.softmax = nn.Softmax()

    # Defining the forward pass    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # print(x.shape)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # x = self.softmax(x)
        return x

net = Net()
optimizer = Adam(net.parameters(), lr=0.00001)
criterion = CrossEntropyLoss()
if torch.cuda.is_available():
    net = net.cuda()
    criterion = criterion.cuda()
device = torch.device('cuda:0')

In [ ]:
#Training Stage
for epoch in tqdm(range(10)):  # loop over the dataset multiple times

    running_loss = 0.0
    running_accuracy = 0.0

    for i, data in enumerate(tqdm(train_dataloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        predictions = net.softmax(outputs).argmax(dim=1)
        # print(labels)
        # labels1 = labels.argmax(dim=1)
        num_correct = torch.sum(predictions == labels).item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        running_accuracy += num_correct/predictions.shape[0]

    print('[%d] loss: %f avg_accuracy: %f' %
              (epoch + 1, running_loss, (running_accuracy/i)))

print('Finished Training')
#minibatch, cnn parameter, epoch, learning rate,   
model_save_name = 'classifier3.pt'
path = F"/content/drive/MyDrive/Machine Learning/{model_save_name}" 
torch.save(net.state_dict(), path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



[1] loss: 2200.200224 avg_accuracy: 0.335943



[2] loss: 2187.682328 avg_accuracy: 0.355671



[3] loss: 2159.872921 avg_accuracy: 0.367885



[4] loss: 2144.100944 avg_accuracy: 0.375315


...


[6] loss: 2118.169297 avg_accuracy: 0.385074



[7] loss: 2105.094285 avg_accuracy: 0.389208



[8] loss: 2094.075328 avg_accuracy: 0.394392



[9] loss: 2083.372288 avg_accuracy: 0.398653



[10] loss: 2072.300557 avg_accuracy: 0.404466

Finished Training
